## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Oct. 13th Oct. 20th


In [57]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [58]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [59]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [60]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [61]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,October-15-2024,Condo/Co-op,3400 S Ocean Blvd Unit 9f,Highland Beach,FL,33487.0,1375000.0,2.0,2.0,Coronado AT Highland Beach Condo,1452.0,NaN,1983.0,NaN,947.0,1750.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3400-...,Beaches MLS,RX-10962844,N,Y,26.415076,-80.065371


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [62]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [63]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
40,PAST SALE,2024-10-18,Condo/Co-op,601 Muirfield Ct Unit 601b,Jupiter,FL,33458.0,300000.0,2.0,2.0,Muirfield Condo,1160.0,NaN,1990.0,NaN,259.0,536.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/601-Muirfiel...,Beaches MLS,RX-11002599,N,Y,26.912540,-80.137756
47,PAST SALE,2024-10-18,Condo/Co-op,107 Lake Dora Dr E,West Palm Beach,FL,33411.0,130000.0,2.0,2.0,Golden Lakes Village Condo 1 Thru 13-a,812.0,NaN,1979.0,NaN,160.0,442.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/107-...,Beaches MLS,RX-11000446,N,Y,26.706611,-80.167397
70,PAST SALE,2024-10-18,Condo/Co-op,4697 Greentree Ter Unit A,Boynton Beach,FL,33436.0,247000.0,2.0,2.0,Greentree Villas,1400.0,NaN,1979.0,NaN,176.0,552.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/4697-G...,Beaches MLS,F10433010,N,Y,26.526156,-80.117959
26,PAST SALE,2024-10-18,Condo/Co-op,502 Osprey Dr Unit 20-C,Delray Beach,FL,33444.0,270000.0,2.0,2.0,Town & Country Estates Condo,1048.0,NaN,1976.0,NaN,258.0,492.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/502-Osp...,Beaches MLS,RX-11016087,N,Y,26.433038,-80.077928
69,PAST SALE,2024-10-18,Condo/Co-op,13403 Touchstone Pl Unit A-105,Palm Beach Gardens,FL,33418.0,305000.0,2.0,2.0,Pinewood Condos,1245.0,NaN,1982.0,NaN,245.0,702.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1340...,Beaches MLS,RX-10979019,N,Y,26.873313,-80.143499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,PAST SALE,2024-10-15,Condo/Co-op,2427 Presidential Way #802,West Palm Beach,FL,33401.0,705000.0,3.0,2.5,Parliament East Condo,2559.0,NaN,1982.0,NaN,275.0,1972.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/2427...,Beaches MLS,RX-11007451,N,Y,26.733165,-80.087255
48,PAST SALE,2024-10-15,Condo/Co-op,11770 Saint Andrews Pl #302,Wellington,FL,33414.0,465000.0,3.0,2.0,ST ANDREWS AT POLO CLUB C,1492.0,NaN,2001.0,NaN,312.0,535.0,Sold,NaN,NaN,https://www.redfin.com/FL/Wellington/11770-Sai...,MARMLS,A11615685,N,Y,26.650507,-80.232161
8,PAST SALE,2024-10-14,Condo/Co-op,3138 Via Poinciana #216,Lake Worth,FL,33467.0,150000.0,2.0,2.0,Grandview AT Spring Lakes Condo,1125.0,NaN,1988.0,NaN,133.0,450.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Worth/3138-Via-...,Beaches MLS,RX-11019115,N,Y,26.630563,-80.154125
49,PAST SALE,2024-10-14,Condo/Co-op,3605 Quail Ridge Dr S Unit Bobwhite A,Boynton Beach,FL,33436.0,517000.0,2.0,2.0,Quail Ridge,1737.0,NaN,1978.0,NaN,298.0,1337.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/3605-N...,Beaches MLS,RX-10972195,N,Y,26.509504,-80.100843


In [64]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [65]:
df_top_ten = df.head(10)

## Collect Agent Information

In [66]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [67]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [68]:
response_df = pd.DataFrame(response_list)

In [69]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [70]:
df_top_ten = merged_df

## Current Week's Values

In [71]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

78


In [72]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$553,339


In [73]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$328


In [74]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$43,160,475


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Palm Beach
Input Previous Week Condo Sales Total (number units sold):
57
Input Previous Week Condo Average Sales Price:
387771
Input Previous Week Condo Average PSF:
276
Input Previous Week Condo Sales total (ex: 198_000_000)
45_900_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [75]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [76]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [77]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [78]:
pd.set_option('display.max_columns',None)

In [79]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-18,Condo/Co-op,725 S Ocean Blvd #303,Boca Raton,FL,33432.0,9600000.0,4.0,3.5,Boca Beach Residences,4328.0,140089.0,2023.0,NaN,2218.0,5399.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/725-S-Oce...,Beaches MLS,RX-10934441,N,Y,26.341326,-80.072755,https://www.redfin.com/FL/Boca-Raton/725-S-Oce...,Jacqueline Feldman,One Sotheby's International Realty,None,None,Lisa Bailey Levy,EXP Realty LLC,None,None,1,orange


In [80]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [81]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-18,Condo/Co-op,725 S Ocean Blvd #303,Boca Raton,FL,33432.0,9600000.0,4.0,3.5,Boca Beach Residences,4328.0,140089.0,2023.0,NaN,2218.0,5399.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/725-S-Oce...,Beaches MLS,RX-10934441,N,Y,26.341326,-80.072755,https://www.redfin.com/FL/Boca-Raton/725-S-Oce...,Jacqueline Feldman,One Sotheby's International Realty,None,None,Lisa Bailey Levy,EXP Realty LLC,None,None,1,orange
1,PAST SALE,2024-10-17,Condo/Co-op,2600 S Ocean Blvd Unit 101n,Palm Beach,FL,33480.0,3600000.0,2.0,2.5,Twenty Six Hundred Condo,2145.0,NaN,1977.0,NaN,1678.0,2520.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2600-S-Oc...,Beaches MLS,RX-10994324,N,Y,26.625982,-80.037408,https://www.redfin.com/FL/Palm-Beach/2600-S-Oc...,Jacqueline Zimmerman,Douglas Elliman (Palm Beach),Adam Zimmerman,Douglas Elliman (Palm Beach),Alyse Keller,Compass Florida LLC (PB),Justin Keller,Compass Florida LLC (PB),2,blue
2,PAST SALE,2024-10-16,Condo/Co-op,3740 S Ocean Blvd #1208,Highland Beach,FL,33487.0,3100000.0,3.0,3.5,Toscana South,2842.0,NaN,2004.0,NaN,1091.0,2696.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3740-...,Beaches MLS,RX-11001019,N,Y,26.407547,-80.065582,https://www.redfin.com/FL/Highland-Beach/3740-...,Judith Randon,Judith Randon Realty,Nancy Ghen,Coldwell Banker/BR,Daniel Gagnon,RE/MAX Select Group,None,None,3,blue
3,PAST SALE,2024-10-15,Condo/Co-op,350 S Ocean Blvd Unit 2-A,Boca Raton,FL,33432.0,2300000.0,3.0,3.0,Beresford Condo,2523.0,NaN,1977.0,NaN,912.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-10954804,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Margaret Oliphant,Atlantic Florida Properties Inc,None,None,Renee Shine,Lang Realty/ BR,None,None,4,blue
4,PAST SALE,2024-10-15,Condo/Co-op,11540 Villa Vasari Dr,Palm Beach Gardens,FL,33418.0,1550000.0,2.0,2.0,Mirasol Villa Vasari,2221.0,NaN,2004.0,NaN,698.0,1889.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/1...,Beaches MLS,RX-11007193,N,Y,26.847560,-80.156123,https://www.redfin.com/FL/Palm-Beach-Gardens/1...,Carol Falciano,Mirasol Realty,None,None,Victoria Rosenfield,"Leibowitz Realty Group, LLC./PBG",None,None,5,blue
5,PAST SALE,2024-10-15,Condo/Co-op,3400 S Ocean Blvd Unit 9f,Highland Beach,FL,33487.0,1375000.0,2.0,2.0,Coronado AT Highland Beach Condo,1452.0,NaN,1983.0,NaN,947.0,1750.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3400-...,Beaches MLS,RX-10962844,N,Y,26.415076,-80.065371,https://www.redfin.com/FL/Highland-Beach/3400-...,None,Coldwell Banker,None,None,Marie Caradonna,Coldwell Banker Realty /Delray Beach,None,None,6,blue
6,PAST SALE,2024-10-17,Condo/Co-op,226 Tresana Blvd #58,Jupiter,FL,33478.0,906000.0,2.0,2.0,Jupiter Country Club,2124.0,NaN,2013.0,NaN,427.0,1293.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/226-Tresana-...,Beaches MLS,RX-11017361,N,Y,26.926339,-80.156486,https://www.redfin.com/FL/Jupiter/226-Tresana-...,Sam Elias,The Agency,None,None,Andrewe Guenther,Coldwell Banker Realty,None,None,7,blue
7,PAST SALE,2024-10-15,Condo/Co-op,1225 S Ocean Blvd #303,Delray Beach,FL,33483.0,875000.0,1.0,1.5,Banyan House Condo,1066.0,NaN,1967.0,NaN,821.0,1086.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Beaches MLS,RX-10997604,N,Y,26.443001,-80.061098,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Susan Risdon,Coldwell Banker Realty /Delray Beach,Curtis Bennett,Coldwell Banker Realty /Delray Beach,Susan Risdon,Coldwell Banker Realty /Delray Beach,Curtis Bennett,Col

In [82]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [83]:
m.save('index.html')

## Data snagger

In [84]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [85]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-18,Condo/Co-op,725 S Ocean Blvd #303,Boca Raton,FL,33432.0,9600000.0,4.0,3.5,Boca Beach Residences,4328.0,140089.0,2023.0,NaN,2218.0,5399.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/725-S-Oce...,Beaches MLS,RX-10934441,N,Y,26.341326,-80.072755,https://www.redfin.com/FL/Boca-Raton/725-S-Oce...,Jacqueline Feldman,One Sotheby's International Realty,None,None,Lisa Bailey Levy,EXP Realty LLC,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [86]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PBC_County_condo_sales_week_ending_10212024


# CREATE TEMPLATE 

In [87]:
muni_set = set(df_top_ten['CITY'])

In [88]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [89]:
df_top_ten.reset_index(inplace=True,drop=True)

In [90]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [91]:
top_sale

'Boca Beach Residences at 725 S Ocean Blvd #303 in Boca Raton'

In [92]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [93]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [94]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-18,Condo/Co-op,725 S Ocean Blvd #303,Boca Raton,FL,33432.0,9600000.0,4.0,3.5,Boca Beach Residences,4328.0,140089.0,2023.0,NaN,2218.0,5399.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/725-S-Oce...,Beaches MLS,RX-10934441,N,Y,26.341326,-80.072755,https://www.redfin.com/FL/Boca-Raton/725-S-Oce...,Jacqueline Feldman,One Sotheby's International Realty,None,None,Lisa Bailey Levy,EXP Realty LLC,None,None,1,orange,Boca Beach Residences at 725 S Ocean Blvd #303...
1,PAST SALE,2024-10-17,Condo/Co-op,2600 S Ocean Blvd Unit 101n,Palm Beach,FL,33480.0,3600000.0,2.0,2.5,Twenty Six Hundred Condo,2145.0,NaN,1977.0,NaN,1678.0,2520.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2600-S-Oc...,Beaches MLS,RX-10994324,N,Y,26.625982,-80.037408,https://www.redfin.com/FL/Palm-Beach/2600-S-Oc...,Jacqueline Zimmerman,Douglas Elliman (Palm Beach),Adam Zimmerman,Douglas Elliman (Palm Beach),Alyse Keller,Compass Florida LLC (PB),Justin Keller,Compass Florida LLC (PB),2,blue,Twenty Six Hundred Condo at 2600 S Ocean Blvd ...
2,PAST SALE,2024-10-16,Condo/Co-op,3740 S Ocean Blvd #1208,Highland Beach,FL,33487.0,3100000.0,3.0,3.5,Toscana South,2842.0,NaN,2004.0,NaN,1091.0,2696.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3740-...,Beaches MLS,RX-11001019,N,Y,26.407547,-80.065582,https://www.redfin.com/FL/Highland-Beach/3740-...,Judith Randon,Judith Randon Realty,Nancy Ghen,Coldwell Banker/BR,Daniel Gagnon,RE/MAX Select Group,None,None,3,blue,Toscana South at 3740 S Ocean Blvd #1208 in Hi...
3,PAST SALE,2024-10-15,Condo/Co-op,350 S Ocean Blvd Unit 2-A,Boca Raton,FL,33432.0,2300000.0,3.0,3.0,Beresford Condo,2523.0,NaN,1977.0,NaN,912.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-10954804,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Margaret Oliphant,Atlantic Florida Properties Inc,None,None,Renee Shine,Lang Realty/ BR,None,None,4,blue,Beresford Condo at 350 S Ocean Blvd Unit 2-A i...
4,PAST SALE,2024-10-15,Condo/Co-op,11540 Villa Vasari Dr,Palm Beach Gardens,FL,33418.0,1550000.0,2.0,2.0,Mirasol Villa Vasari,2221.0,NaN,2004.0,NaN,698.0,1889.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/1...,Beaches MLS,RX-11007193,N,Y,26.847560,-80.156123,https://www.redfin.com/FL/Palm-Beach-Gardens/1...,Carol Falciano,Mirasol Realty,None,None,Victoria Rosenfield,"Leibowitz Realty Group, LLC./PBG",None,None,5,blue,Mirasol Villa Vasari at 11540 Villa Vasari Dr ...
5,PAST SALE,2024-10-15,Condo/Co-op,3400 S Ocean Blvd Unit 9f,Highland Beach,FL,33487.0,1375000.0,2.0,2.0,Coronado AT Highland Beach Condo,1452.0,NaN,1983.0,NaN,947.0,1750.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3400-...,Beaches MLS,RX-10962844,N,Y,26.415076,-80.065371,https://www.redfin.com/FL/Highland-Beach/3400-...,None,Coldwell Banker,None,None,Marie Caradonna,Coldwell Banker Realty /Delray Beach,None,None,6,blue,Coronado AT Highland Beach Condo at 3400 S Oce...
6,PAST SALE,2024-10-17,Condo/Co-op,226 Tresana Blvd #58,Jupiter,FL,33478.0,906000.0,2.0,2.0,Jupiter Country Club,2124.0,NaN,2013.0,NaN,427.0,1293.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/226-Tresana-...,Beaches MLS,RX-11017361,N,Y,26.926339,-80.156486,https://www.redfin.com/FL/Jupiter/226-Tresana-...,Sam Elias,The Agency,None,None,Andrewe Guenther,Coldwell Banker Realty,None,None,7,blue,Jupiter Country Club at 226 Tresana Blvd #58 i...
7,PAST SALE,2024-10-15,Condo/Co-op,1225 S Ocean Blvd #303,Delray Beach,FL,33483.0,875000.0,1.0,1.5,Banyan House Condo,1066.0,

In [95]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Boca Beach Residences closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $705,000M to $9,600,000M
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Highland Beach, Delray Beach, Jupiter, Palm Beach Gardens, Palm Beach, Boca Raton, West Palm Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 78 condo sales totaling $43,160,475 million from Oct. 13th to Oct. 20th. The previous week, brokers closed 57 condo sales totaling $45,900,000.

Last week’s units sold for an average of $553,339, higher than the $387,771 average price f

In [96]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [97]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [98]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [99]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-18,Condo/Co-op,725 S Ocean Blvd #303,Boca Raton,FL,33432.0,9600000.0,4.0,3.5,Boca Beach Residences,4328.0,140089.0,2023.0,NaN,2218.0,5399.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/725-S-Oce...,Beaches MLS,RX-10934441,N,Y,26.341326,-80.072755,https://www.redfin.com/FL/Boca-Raton/725-S-Oce...,Jacqueline Feldman,One Sotheby's International Realty,None,None,Lisa Bailey Levy,EXP Realty LLC,None,None,1,orange,Boca Beach Residences at 725 S Ocean Blvd #303...
1,PAST SALE,2024-10-17,Condo/Co-op,2600 S Ocean Blvd Unit 101n,Palm Beach,FL,33480.0,3600000.0,2.0,2.5,Twenty Six Hundred Condo,2145.0,NaN,1977.0,NaN,1678.0,2520.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2600-S-Oc...,Beaches MLS,RX-10994324,N,Y,26.625982,-80.037408,https://www.redfin.com/FL/Palm-Beach/2600-S-Oc...,Jacqueline Zimmerman,Douglas Elliman (Palm Beach),Adam Zimmerman,Douglas Elliman (Palm Beach),Alyse Keller,Compass Florida LLC (PB),Justin Keller,Compass Florida LLC (PB),2,blue,Twenty Six Hundred Condo at 2600 S Ocean Blvd ...
2,PAST SALE,2024-10-16,Condo/Co-op,3740 S Ocean Blvd #1208,Highland Beach,FL,33487.0,3100000.0,3.0,3.5,Toscana South,2842.0,NaN,2004.0,NaN,1091.0,2696.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3740-...,Beaches MLS,RX-11001019,N,Y,26.407547,-80.065582,https://www.redfin.com/FL/Highland-Beach/3740-...,Judith Randon,Judith Randon Realty,Nancy Ghen,Coldwell Banker/BR,Daniel Gagnon,RE/MAX Select Group,None,None,3,blue,Toscana South at 3740 S Ocean Blvd #1208 in Hi...
3,PAST SALE,2024-10-15,Condo/Co-op,350 S Ocean Blvd Unit 2-A,Boca Raton,FL,33432.0,2300000.0,3.0,3.0,Beresford Condo,2523.0,NaN,1977.0,NaN,912.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-10954804,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Margaret Oliphant,Atlantic Florida Properties Inc,None,None,Renee Shine,Lang Realty/ BR,None,None,4,blue,Beresford Condo at 350 S Ocean Blvd Unit 2-A i...
4,PAST SALE,2024-10-15,Condo/Co-op,11540 Villa Vasari Dr,Palm Beach Gardens,FL,33418.0,1550000.0,2.0,2.0,Mirasol Villa Vasari,2221.0,NaN,2004.0,NaN,698.0,1889.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/1...,Beaches MLS,RX-11007193,N,Y,26.847560,-80.156123,https://www.redfin.com/FL/Palm-Beach-Gardens/1...,Carol Falciano,Mirasol Realty,None,None,Victoria Rosenfield,"Leibowitz Realty Group, LLC./PBG",None,None,5,blue,Mirasol Villa Vasari at 11540 Villa Vasari Dr ...
5,PAST SALE,2024-10-15,Condo/Co-op,3400 S Ocean Blvd Unit 9f,Highland Beach,FL,33487.0,1375000.0,2.0,2.0,Coronado AT Highland Beach Condo,1452.0,NaN,1983.0,NaN,947.0,1750.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3400-...,Beaches MLS,RX-10962844,N,Y,26.415076,-80.065371,https://www.redfin.com/FL/Highland-Beach/3400-...,None,Coldwell Banker,None,None,Marie Caradonna,Coldwell Banker Realty /Delray Beach,None,None,6,blue,Coronado AT Highland Beach Condo at 3400 S Oce...
6,PAST SALE,2024-10-17,Condo/Co-op,226 Tresana Blvd #58,Jupiter,FL,33478.0,906000.0,2.0,2.0,Jupiter Country Club,2124.0,NaN,2013.0,NaN,427.0,1293.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/226-Tresana-...,Beaches MLS,RX-11017361,N,Y,26.926339,-80.156486,https://www.redfin.com/FL/Jupiter/226-Tresana-...,Sam Elias,The Agency,None,None,Andrewe Guenther,Coldwell Banker Realty,None,None,7,blue,Jupiter Country Club at 226 Tresana Blvd #58 i...
7,PAST SALE,2024-10-15,Condo/Co-op,1225 S Ocean Blvd #303,Delray Beach,FL,33483.0,875000.0,1.0,1.5,Banyan House Condo,1066.0,

In [100]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: Boca Beach Residences closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $705,000M to $9,600,000M
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Highland Beach, Delray Beach, Jupiter, Palm Beach Gardens, Palm Beach, Boca Raton, West Palm Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 78 condo sales totaling $43,160,475 million from Oct. 13th to Oct. 20th. The previous week, brokers closed 57 condo sales totaling $45,900,000.

Last week’s units sold for an average of $553,339, higher than the $387,771 average price f

In [101]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Boca-Raton/725-S-Ocean-Blvd-33432/unit-303/home/188554171


In [102]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Palm-Beach/2600-S-Ocean-Blvd-33480/unit-101N/home/42436541


In [103]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/West-Palm-Beach/2427-Presidential-Way-33401/unit-802/home/42516927


In [104]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Highland-Beach/3400-S-Ocean-Blvd-33487/unit-4A/home/42394528


In [105]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Boca-Raton/725-S-Ocean-Blvd-33432/unit-303/home/188554171


In [106]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/West-Palm-Beach/2427-Presidential-Way-33401/unit-802/home/42516927


In [107]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-18,Condo/Co-op,725 S Ocean Blvd #303,Boca Raton,FL,33432.0,9600000.0,4.0,3.5,Boca Beach Residences,4328.0,140089.0,2023.0,NaN,2218.0,5399.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/725-S-Oce...,Beaches MLS,RX-10934441,N,Y,26.341326,-80.072755,https://www.redfin.com/FL/Boca-Raton/725-S-Oce...,Jacqueline Feldman,One Sotheby's International Realty,None,None,Lisa Bailey Levy,EXP Realty LLC,None,None,1,orange,Boca Beach Residences at 725 S Ocean Blvd #303...
1,PAST SALE,2024-10-17,Condo/Co-op,2600 S Ocean Blvd Unit 101n,Palm Beach,FL,33480.0,3600000.0,2.0,2.5,Twenty Six Hundred Condo,2145.0,NaN,1977.0,NaN,1678.0,2520.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2600-S-Oc...,Beaches MLS,RX-10994324,N,Y,26.625982,-80.037408,https://www.redfin.com/FL/Palm-Beach/2600-S-Oc...,Jacqueline Zimmerman,Douglas Elliman (Palm Beach),Adam Zimmerman,Douglas Elliman (Palm Beach),Alyse Keller,Compass Florida LLC (PB),Justin Keller,Compass Florida LLC (PB),2,blue,Twenty Six Hundred Condo at 2600 S Ocean Blvd ...
2,PAST SALE,2024-10-16,Condo/Co-op,3740 S Ocean Blvd #1208,Highland Beach,FL,33487.0,3100000.0,3.0,3.5,Toscana South,2842.0,NaN,2004.0,NaN,1091.0,2696.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3740-...,Beaches MLS,RX-11001019,N,Y,26.407547,-80.065582,https://www.redfin.com/FL/Highland-Beach/3740-...,Judith Randon,Judith Randon Realty,Nancy Ghen,Coldwell Banker/BR,Daniel Gagnon,RE/MAX Select Group,None,None,3,blue,Toscana South at 3740 S Ocean Blvd #1208 in Hi...
3,PAST SALE,2024-10-15,Condo/Co-op,350 S Ocean Blvd Unit 2-A,Boca Raton,FL,33432.0,2300000.0,3.0,3.0,Beresford Condo,2523.0,NaN,1977.0,NaN,912.0,4279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-10954804,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Margaret Oliphant,Atlantic Florida Properties Inc,None,None,Renee Shine,Lang Realty/ BR,None,None,4,blue,Beresford Condo at 350 S Ocean Blvd Unit 2-A i...
4,PAST SALE,2024-10-15,Condo/Co-op,11540 Villa Vasari Dr,Palm Beach Gardens,FL,33418.0,1550000.0,2.0,2.0,Mirasol Villa Vasari,2221.0,NaN,2004.0,NaN,698.0,1889.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/1...,Beaches MLS,RX-11007193,N,Y,26.847560,-80.156123,https://www.redfin.com/FL/Palm-Beach-Gardens/1...,Carol Falciano,Mirasol Realty,None,None,Victoria Rosenfield,"Leibowitz Realty Group, LLC./PBG",None,None,5,blue,Mirasol Villa Vasari at 11540 Villa Vasari Dr ...
5,PAST SALE,2024-10-15,Condo/Co-op,3400 S Ocean Blvd Unit 9f,Highland Beach,FL,33487.0,1375000.0,2.0,2.0,Coronado AT Highland Beach Condo,1452.0,NaN,1983.0,NaN,947.0,1750.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3400-...,Beaches MLS,RX-10962844,N,Y,26.415076,-80.065371,https://www.redfin.com/FL/Highland-Beach/3400-...,None,Coldwell Banker,None,None,Marie Caradonna,Coldwell Banker Realty /Delray Beach,None,None,6,blue,Coronado AT Highland Beach Condo at 3400 S Oce...
6,PAST SALE,2024-10-17,Condo/Co-op,226 Tresana Blvd #58,Jupiter,FL,33478.0,906000.0,2.0,2.0,Jupiter Country Club,2124.0,NaN,2013.0,NaN,427.0,1293.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/226-Tresana-...,Beaches MLS,RX-11017361,N,Y,26.926339,-80.156486,https://www.redfin.com/FL/Jupiter/226-Tresana-...,Sam Elias,The Agency,None,None,Andrewe Guenther,Coldwell Banker Realty,None,None,7,blue,Jupiter Country Club at 226 Tresana Blvd #58 i...
7,PAST SALE,2024-10-15,Condo/Co-op,1225 S Ocean Blvd #303,Delray Beach,FL,33483.0,875000.0,1.0,1.5,Banyan House Condo,1066.0,

## Time on Market Calculator

In [56]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 5, 7) ## List (Earlier) date
date2 = datetime(2024, 10, 17) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

163


## Clean CSV for Datawrapper Chart

In [108]:
chart_df = df_top_ten

In [109]:
chart_df = chart_df.fillna(" ")

In [110]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [111]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [112]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [113]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [114]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [115]:
csv_date_string = today.strftime("%m_%d_%Y")

In [116]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")